<a href="https://colab.research.google.com/github/kitlapp/HotelBookingSQLPreprocessing/blob/kimon/PythonPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hotel Booking Data Preprocessing in the Google Colab Platform

The goal of this project is to migrate the LookerStudioKPIDashboard project, originally prepared in a local environment, to the cloud, specifically to Google Colab. This notebook therefore includes all Python scripts required to create a cleaned DataFrame, suitable for KPI calculations for booking cancellation monitoring and further BI processing.

The preprocessing in this approach is slightly different from the one used for machine learning purposes (BookingCancellationPrediction). For example, null values must be handled in both DataFrames, since BI tools cannot work with them either. However, in cases such as dates, the KPI-focused DataFrame does not require trigonometric component calculations to encode cyclical patterns. Instead, it only needs dates in the correct format, e.g., "YYYY/MM/DD".

The advantages of this migration are:

1. A local environment setup is not required (e.g., connecting to a Jupyter server, managing the command line through Conda, keeping command history logs for reproducible setup, or using Git Bash for version control). Even a gitignore file is unnecessary, since one can carefully choose what to upload directly to this specific GitHub repository from their PC.
2. Working in Google Colab and potentially purchasing resources in the future makes this approach scalable.
3. This environment can be shared and promote collaboration more easily, because replicating the project on any machine only requires cloning the GitHub repository URL in Google Colab. From there, one can immediately run the scripts or start enhancing the code.
4. Google Colab has Gemini integrated which enhances coding help.



# 1. Importing Libraries and Reading the Raw Data from Google Drive

In [4]:
# Authorize access of Google Colab to Google Drive
# The code below has to be executed only once
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Google Drive path to the dataset
filepath = '/content/drive/MyDrive/PyCharm_Projects/hotel_booking_RAW.csv'

# Read csv file to a DataFrame
df_raw = pd.read_csv(filepath)

# Rows and columns check of raw data (For SQL preprocessing comparison)
print(df_raw.shape)

# Check for duplicates
print('Number of Duplicates:', df_raw.duplicated().sum())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(119390, 36)
Number of Duplicates: 0


# 2. Preprocessing

## 2.1. Handling Null Values

In [ ]:
# Check for nulls
df_raw.isna().sum()

In [6]:
# Make a copy before further preprocessing ('dash': Dashboard, to link with the purpose of this preprocessing)
dfdash2 = df_raw.copy()

# Fill missing values in 'children' column with the most frequent value (mode)
dfdash2['children'] = dfdash2['children'].fillna(value=dfdash2['children'].mode()[0])

# Replace missing values in 'agent' with 0, indicating direct bookings without a travel agent
dfdash2['agent'] = dfdash2['agent'].fillna(value=0)

# Replace missing values in 'company' with 0, meaning bookings not linked to any company
dfdash2['company'] = dfdash2['company'].fillna(value=0)

# Drop all rows with missing 'country' values since location info is important for analysis
dfdash2 = dfdash2.drop(labels=dfdash2.loc[dfdash2['country'].isna()].index)

# Rows and columns check of dfdash2 (For SQL preprocessing comparison)
print(dfdash2.shape)

(118902, 36)


## 2.2. Handling Date-Related Columns

In [7]:
# Make a copy before further preprocessing
dfdash3 = dfdash2.copy()

# Create a dictionary to convert month names to their corresponding numeric values (as strings)
month_mapping = {
    "January": '1', "February": '2', "March": '3', "April": '4', "May": '5',
    "June": '6', "July": '7', "August": '8', "September": '9', "October": '10',
    "November": '11', "December": '12'
}

# Map month names to integers using the same dictionary
dfdash3['arrival_date_month'] = dfdash3['arrival_date_month'].map(month_mapping).astype(int)

# Combine year, month, and day columns into a single date string in 'YYYY-MM-DD' format
dfdash3['arrival_date'] = (
    dfdash3['arrival_date_year'].astype(str) + '-' +
    dfdash3['arrival_date_month'].astype(str) + '-' +
    dfdash3['arrival_date_day_of_month'].astype(str)
)

# Convert the date strings into proper datetime objects
# for easier time-based analysis
dfdash3['arrival_date'] = pd.to_datetime(dfdash3['arrival_date'], format='%Y-%m-%d')

# Rows and columns check of dfdash3 (For SQL preprocessing comparison)
print(dfdash3.shape)

(118902, 37)


## 2.3. Dropping Unimportant Columns

In [10]:
# List of columns to be dropped from the dashboard dataframe
dashcols_to_be_dropped = ['name', 'email', 'arrival_date_month', 'arrival_date_day_of_month', 'phone-number',
                          'credit_card', 'reservation_status', 'reservation_status_date', 'assigned_room_type',
                          'deposit_type', 'required_car_parking_spaces', 'arrival_date_week_number']

# Make a copy before further preprocessing
dfdash4 = dfdash3.copy()

# Drop the unimportant columns from the dashboard dataframe
dfdash4 = dfdash4.drop(columns=dashcols_to_be_dropped)

# Rows and columns check of dfdash4 (For SQL preprocessing comparison)
print(dfdash4.shape)

(118902, 25)


## 2.4. Creating total_kids Column

In [11]:
# Make a copy before further preprocessing
dfdash5 = dfdash4.copy()

# Merge the 'children' and 'babies' columns to create a new column 'total_kids' representing the total number of kids
dfdash5['total_kids'] = dfdash5['children'].astype(int) + dfdash5['babies'].astype(int)

# Drop the original 'children' and 'babies' columns after merging
dfdash5 = dfdash5.drop(columns=['children', 'babies'])

# Drop rows with outliers (total kids > 3) and reset index in the dashboard dataframe
dfdash5 = dfdash5.loc[dfdash5['total_kids'] <= 3].reset_index(drop=True)

# Rows and columns check of dfdash5 (For SQL preprocessing comparison)
print(dfdash5.shape)

(118899, 24)


## 2.5. Handling adults Column

There are observations where both adults and total_kids equal 0. This can't be explained and therefore all rows where adults = 0 will be dropped. Additionally, in all cases where adults were greater than 4 the bookings were canceled and the adr equals 0. For this reason, values for adults from 1 to 4 are considered the most explainable and normal. We will drop all other values.

In [13]:
# Make a copy before further preprocessing
dfdash6 = dfdash5.copy()

# Exclude bookings where the number of adults is 0. Also, ensure that the number of adults is between 1 and 4.
dfdash6 = dfdash6[(dfdash6['adults'] > 0) & (dfdash6['adults'] <= 4)].reset_index(drop=True)

# Rows and columns check of dfdash6 (For SQL preprocessing comparison)
print(dfdash6.shape)

(118490, 24)


## 2.6. Handling meal Column

In [15]:
# Make a copy before further preprocessing
dfdash7 = dfdash6.copy()

# Create a dictionary to map meal types to numerical values
meal_mapping = {'BB': 1, 'HB': 2, 'SC': 0, 'FB': 3}

# Drop rows where the 'meal' column is 'Undefined', indicating no meal choice
dfdash7 = dfdash7.drop(labels=dfdash7[dfdash7['meal'] == 'Undefined'].index).reset_index(drop=True)

# Rename the 'meal' column to 'number_of_meals' for clarity
dfdash7 = dfdash7.rename(columns={'meal': 'number_of_meals'})

# Map the dictionary to the 'number_of_meals' column, reducing complexity
dfdash7['number_of_meals'] = dfdash7['number_of_meals'].map(meal_mapping).astype(int)

# Rows and columns check of dfdash7 (For SQL preprocessing comparison)
print(dfdash7.shape)

(117325, 24)


## 2.7. Handling market_segment Column

In [16]:
# Make a copy before further preprocessing
dfdash8 = dfdash7.copy()

# Drop all rows where the 'market_segment' column has the category 'Undefined'
dfdash8 = dfdash8.drop(labels=dfdash8[dfdash8['market_segment'] == 'Undefined'].index).reset_index(drop=True)

# Replace the 'Complementary' and 'Aviation' categories in the 'market_segment' column with 'Other'
dfdash8['market_segment'] = dfdash8['market_segment'].replace(
    {'Complementary': 'Other', 'Aviation': 'Other'}).astype('category')

# Rows and columns check of dfdash8 (For SQL preprocessing comparison)
print(dfdash8.shape)

(117323, 24)


## 2.8. Handling Distribution Channel Column

In [17]:
# Make a copy before further preprocessing
dfdash9 = dfdash8.copy()

# Drop all rows where the 'distribution_channel' column has the category 'Undefined'
dfdash9 = dfdash9.drop(labels=dfdash9[dfdash9['distribution_channel'] == 'Undefined'].index).reset_index(drop=True)

# Convert the 'distribution_channel' column to categorical type
dfdash9['distribution_channel'] = dfdash9['distribution_channel'].astype('category')

# Rows and columns check of dfdash9 (For SQL preprocessing comparison)
print(dfdash9.shape)

(117320, 24)


## 2.9. Handling reserved_room_type Column

In [18]:
# Make a copy before further preprocessing
dfdash10 = dfdash9.copy()

# Merge categories in the 'reserved_room_type' column, combining multiple categories into 'Other'
dfdash10['reserved_room_type'] = dfdash10['reserved_room_type'].replace(
    {'C': 'Other', 'B': 'Other', 'H': 'Other', 'L': 'Other'}).astype('category')

# Rows and columns check of dfdash9 (For SQL preprocessing comparison)
print(dfdash10.shape)

(117320, 24)


## 2.10. Handling agent & company Columns

In [19]:
# Make a copy before further preprocessing
dfdash11 = dfdash10.copy()

# Convert 'agent' column to binary: 1 if not 0, else 0
dfdash11['agent'] = dfdash11['agent'].apply(lambda x: 1 if x != 0 else 0)

# Convert 'company' column to binary: 1 if not 0, else 0
dfdash11['company'] = dfdash11['company'].apply(lambda x: 1 if x != 0 else 0)

# Rename the columns to more intuitive names
dfdash11 = dfdash11.rename(columns={'agent': 'has_agent', 'company': 'has_company'})

# Rows and columns check of dfdash9 (For SQL preprocessing comparison)
print(dfdash11.shape)

(117320, 24)


## 2.11. Handling adr & lead_time Outliers

In [21]:
# Make a copy before further preprocessing
dfdash12 = dfdash11.copy()

# Set the threshold values for ADR (average daily rate) and lead_time outliers:
adr_outlier_value = 5400  # Maximum acceptable value for ADR
lead_time_outlier_border = 640  # Maximum acceptable value for lead time

# Remove rows where ADR is higher than the defined threshold or negative
dfdash12 = dfdash12.loc[(dfdash12['adr'] < adr_outlier_value) & (dfdash12['adr'] >= 0)].reset_index(drop=True)

# Remove rows where lead_time exceeds the defined threshold in the dashboard dataframe
dfdash12 = dfdash12.loc[dfdash12['lead_time'] < lead_time_outlier_border].reset_index(drop=True)

# Rows and columns check of dfdash12 (For SQL preprocessing comparison)
print(dfdash12.shape)

(117316, 24)
